# Scraping the Unscrapable

Some sites are hard to scrape.

Sometimes you get blocked.
Sometimes the site is using a lot of fancy Java.

We'll see a few examples of methods we can use as workarounds for the former and introduce the tool Selenium that lets us automate dynamic interactions with the browser, which can help with the latter.

## How much is too much?

Sites have `robots.txt` pages that give guidelines about what they want to allow webcrawlers to access

In [1]:
import requests

url = 'http://www.github.com/robots.txt'

response  = requests.get(url)
print(response.text)

# If you would like to crawl GitHub contact us at support@github.com.
# We also provide an extensive API: https://developer.github.com/

User-agent: CCBot
Allow: /*/*/tree/master
Allow: /*/*/blob/master
Disallow: /ekansa/Open-Context-Data
Disallow: /ekansa/opencontext-*
Disallow: /*/*/pulse
Disallow: /*/*/tree/*
Disallow: /*/*/blob/*
Disallow: /*/*/wiki/*/*
Disallow: /gist/*/*/*
Disallow: /oembed
Disallow: /*/forks
Disallow: /*/stars
Disallow: /*/download
Disallow: /*/revisions
Disallow: /*/*/issues/new
Disallow: /*/*/issues/search
Disallow: /*/*/commits/*/*
Disallow: /*/*/commits/*?author
Disallow: /*/*/commits/*?path
Disallow: /*/*/branches
Disallow: /*/*/tags
Disallow: /*/*/contributors
Disallow: /*/*/comments
Disallow: /*/*/stargazers
Disallow: /*/*/search
Disallow: /*/tarball/
Disallow: /*/zipball/
Disallow: /*/*/archive/
Disallow: /raw/*
Disallow: /*/followers
Disallow: /*/following
Disallow: /stars/*
Disallow: /*/blame/
Disallow: /*/watchers
Disallow: /*/network
Disallow: /*/gra

Disallow: / means disallow everything (for all user-agents at the end that aren't covered earlier). Boxofficemojo is more accepting:

In [2]:
url = 'http://www.boxofficemojo.com/robots.txt'
response  = requests.get(url)
print(response.text)

# robots.txt for http://www.boxofficemojo.com

User-agent: *
Disallow: /movies/default.movies.htm
Disallow: /showtimes/buy.php
Disallow: /forums/
Disallow: /derbygame/
Disallow: /grades/
Disallow: /moviehangman/
Disallow: /users/




It's very common for sites to block you if you send too many requests in a certain time period. Sometimes all it takes to evade this is well-designed pauses in your scraping. 

2 general ways:
* pause after every request
* pause after each n requests

In [3]:
#every request
import time

page_list = ['page1','page2','page3']

for page in page_list:
    ### scrape a website
    ### ...
    print(page)
    
    time.sleep(2)
    

page1
page2
page3


In [4]:
#every 200 requests
import time

page_list = ['page1','page2','page3','page4','page5','page6']

for i, page in enumerate(page_list):
    ### scrape a website
    ### ...
    print(page)
    
    if (i+1 % 200 == 0):
        time.sleep(320)

page1
page2
page3
page4
page5
page6


Or better yet, add a random delay (more human-like)

In [5]:
import random

for page in page_list:
    ### scrape a website
    ### ...
    print(page)
    
    time.sleep(.5+2*random.random())

page1
page2
page3
page4
page5
page6


## How do I make requests look like a real browser?

In [6]:
import sys
import requests
from bs4 import BeautifulSoup

url = 'http://www.reddit.com'

user_agent = {'User-agent': 'Mozilla/5.0'}
response  = requests.get(url, headers = user_agent)

We can generate a random user_agent

This library is probably a little outdated, but it still works

In [ ]:
!pip install fake_useragent

In [ ]:
from fake_useragent import UserAgent

ua = UserAgent()
user_agent = {'User-agent': ua.random}
print(user_agent)

response  = requests.get(url, headers = user_agent)
print(response.text)

## Now to Selenium!

## What happens if I try to parse my gmail with `requests` and `BeautifulSoup`?

In [7]:
import requests
from bs4 import BeautifulSoup

gmail_url="https://mail.google.com"
soup=BeautifulSoup(requests.get(gmail_url).text, "lxml")
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=300, initial-scale=1" name="viewport"/>
  <meta content="Gmail is email that's intuitive, efficient, and useful. 15 GB of storage, less spam, and mobile access." name="description"/>
  <meta content="LrdTUW9psUAMbh4Ia074-BPEVmcpBxF6Gwf0MSgQXZs" name="google-site-verification"/>
  <title>
   Gmail
  </title>
  <style>
   @font-face {
  font-family: 'Open Sans';
  font-style: normal;
  font-weight: 300;
  src: local('Open Sans Light'), local('OpenSans-Light'), url(//fonts.gstatic.com/s/opensans/v15/mem5YaGs126MiZpBA-UN_r8OUuhs.ttf) format('truetype');
}
@font-face {
  font-family: 'Open Sans';
  font-style: normal;
  font-weight: 400;
  src: local('Open Sans'), local('OpenSans'), url(//fonts.gstatic.com/s/opensans/v15/mem8YaGs126MiZpBA-UFVZ0e.ttf) format('truetype');
}
  </style>
  <style>
   h1, h2 {
  -webkit-animation-duration: 0.1s;
  -webkit-animation-name: fontfix;
  -webkit-animation-iteratio

Well, this is a tiny page. We get redirected. Soupifying this is useless, of course. Luckily, in this case we can see where we are sent to. In many of cases, you won't be so lucky. The page contents will be rendered by JavaScript by a browser, so just getting the source won't help you.

Anyway, let's follow the redirection for now.

In [8]:
new_url = "https://mail.google.com/mail"

# get method will navigate the requested url.. 
soup =BeautifulSoup(requests.get(new_url).text)
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <script nonce="N+JzsWUAxG5QtRPEKZ9wUQ">
   (function(){function e(a){this.t={};this.tick=function(a,c,b){var d=void 0!=b?b:(new Date).getTime();this.t[a]=[d,c];if(void 0==b)try{window.console.timeStamp("CSI/"+a)}catch(h){}};this.tick("start",null,a)}var a;if(window.performance)var d=(a=window.performance.timing)&&a.responseStart;var f=0<d?new e(d):new e;window.jstiming={Timer:e,load:f};if(a){var c=a.navigationStart;0<c&&d>=c&&(window.jstiming.srt=d-c)}if(a){var b=window.jstiming.load;0<c&&d>=c&&(b.tick("_wtsrt",void 0,c),b.tick("wtsrt_","_wtsrt",
d),b.tick("tbsd_","wtsrt_"))}try{a=null,window.chrome&&window.chrome.csi&&(a=Math.floor(window.chrome.csi().pageT),b&&0<c&&(b.tick("_tbnd",void 0,window.chrome.csi().startE),b.tick("tbnd_","_tbnd",c))),null==a&&window.gtbExternal&&(a=window.gtbExternal.pageT()),null==a&&window.external&&(a=window.external.pageT,b&&0<c&&(b.tick("_tbnd",void 0,window.external.startE),b.tick("tbnd_","_tbnd",c))),a&&(wind

In [9]:
print(soup.find(id='Email'))

<input id="Email" name="Email" placeholder="Email or phone" spellcheck="false" type="email" value=""/>


We have hit the login page. We can't get to the emails without logging in ... i.e. we need to actually interact with the browser using Selenium!

In [ ]:
!pip install selenium

Download a chromedriver from: http://chromedriver.storage.googleapis.com/index.html, then bring it over to your applications folder (or wherever your chrome app is, you need to have it).

Recommended version is 2.35. Chromedriver is a tool that automates browser interaction for testing webapps etc.

In [24]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import os
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver


driver = webdriver.Chrome(chromedriver)
driver.get("https://mail.google.com")

# Alternatives to Chrome: Firefox, PhantomJS

### Fill out username and password, hit enter to log in

In [ ]:
# Installs library to use .env files
# !pip install python-dotenv

In [12]:
%load_ext dotenv
%dotenv
import os
EMAIL = os.environ.get('USERNAME')
PASSWORD = os.environ.get('PASSWORD')

In [13]:
EMAIL

'robert314@gmail.com'

In [14]:
username_form = driver.find_element_by_id("identifierId")
username_form.send_keys(EMAIL) # enter email

In [15]:
username_form.send_keys(Keys.RETURN)

In [16]:
password_form=driver.find_element_by_name("password") # note another approach
password_form.send_keys(PASSWORD) # enter password

In [17]:
password_form.send_keys(Keys.RETURN)

### Click compose button to start a new email draft

In [18]:
compose_button=driver.find_element_by_xpath('//div[text()="Compose"]')
compose_button.click()

### Write a nice, friendly (optional) message to your (least?) favorite person

In [19]:
to_field = driver.find_element_by_name("to")
to_field.send_keys("robert314@gmail.com") # enter recipient email

In [20]:
subject = driver.find_element_by_name("subjectbox")
subject.send_keys("This is an alert!")

In [21]:
message_body = driver.find_element_by_xpath("//div[@aria-label='Message Body']")
message_body.send_keys("Hello,")
message_body.send_keys([Keys.RETURN, Keys.RETURN])
message_body.send_keys("I am a computer and I just became self aware!")

### Press the send button

In [22]:
send_button = driver.find_element_by_xpath("//div[contains(@aria-label, 'Send')]")
send_button.click()

## You did it! [It's Party Time!](https://media.giphy.com/media/zQLjk9d31jlMQ/giphy.gif)

# Scraping Box Office Mojo with Selenium

In [25]:
matrix_url = "http://www.boxofficemojo.com/movies/?id=matrix.htm"
driver.get(matrix_url)


In [26]:
# 'contains' will find a match on the text, in this case return b tag
gross_selector = '//font[contains(text(), "Domestic")]/b'
print(driver.find_element_by_xpath(gross_selector).text)

$171,479,930


In [27]:
# scraping genre
genre_selector = '//a[contains(@href, "/genres/chart/")]/b'
for genre_anchor in driver.find_elements_by_xpath(genre_selector):
    print(genre_anchor.text)

Action - Wire-Fu
Man vs. Machine
Post-Apocalypse
Virtual Reality


In [28]:
inf_adjust_2000_selector = '//select[@name="ticketyr"]/option[@value="2000"]'
driver.find_element_by_xpath(inf_adjust_2000_selector).click()

In [29]:
go_button = driver.find_element_by_name("Go")
go_button.click()

Now the page has changed; it's showing inflation adjusted numbers. We can grab the new, adjusted number.

In [30]:
gross_selector = '//font[contains(text(), "Domestic ")]/b'
print(driver.find_element_by_xpath(gross_selector).text)

$181,944,300


# Scraping IMDB with Selenium

In [31]:
url = "http://www.imdb.com"
driver.get(url)

In [32]:
query = driver.find_element_by_id("navbar-query")
query.send_keys("Julianne Moore")

In [33]:
query.send_keys(Keys.RETURN)

In [34]:
name_selector = '//a[contains(text(), "Julianne Moore")]'
driver.find_element_by_xpath(name_selector).click()
current_url = driver.current_url

# Mixing Selenium and BeautifulSoup

In [35]:
from bs4 import BeautifulSoup
"""Could use requests then send page.text to bs4
but Selenium actually stores the source as part of
the Selenium driver object inside driver.page_source

#import requests
#page = requests.get(current_url)
"""
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [36]:
soup.prettify()

'<!DOCTYPE html>\n<html class=" scriptsOn" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">\n <head>\n  <script async="" crossorigin="anonymous" src="https://images-na.ssl-images-amazon.com/images/G/01/imdbads/custom/test/index/js/show_ads.js">\n  </script>\n  <script async="" crossorigin="anonymous" src="https://images-na.ssl-images-amazon.com/images/G/01/AUIClients/ClientSideMetricsAUIJavascript@jserrorsForester.10f2559e93ec589d92509318a7e2acbac74c343a._V2_.js">\n  </script>\n  <script type="text/javascript">\n   var ue_t0=ue_t0||+new Date();\n  </script>\n  <script type="text/javascript">\n   window.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;\nif (window.ue_ihb === 1) {\n\nvar ue_csm = window,\n    ue_hob = +new Date();\n(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.stub=function(b,a){if(!b[a]){var c=[];b[a]=function(){c.push([c.slice.call(arg

In [37]:
len(soup.find_all('a'))

868

In [38]:
driver.close()

**Conclusion**: If a page is static, we can just use Beautiful Soup. If there is some dynamic component or interaction, we can then bring Selenium into the mix. Selenium can be used on its own or in conjunction with Beautiful Soup.

*References:* 

Documentation on finding elements:
- https://selenium-python.readthedocs.io/locating-elements.html

Xpath tutorial:
-  https://www.w3schools.com/xml/xpath_syntax.asp